# Object Detection with camera

SPDX-License-Identifier: MIT  
SPDX-FileCopyrightText: 2022 Daisuke Sato

https://github.com/Tiryoh/mini-pupper-jupyter-notebooks

## Reference

* https://github.com/EdjeElectronics/TensorFlow-Lite-Object-Detection-on-Android-and-Raspberry-Pi

## Set Network configuration

In [ ]:
# NETWORK_INTERFACE="eth0"
NETWORK_INTERFACE="wlan0"

import netifaces as ni
ip_address = ni.ifaddresses(NETWORK_INTERFACE)[ni.AF_INET][0]['addr']
print(ip_address)

In [ ]:
import os

# Set self ip address
os.environ['ROS_IP'] = ip_address

# Set ROS_MASTER_URI
ROS_MASTER_IP=ip_address
# ROS_MASTER_IP="192.168.10.14" # modify here to specify custom ip

os.environ['ROS_MASTER_URI'] = 'http://{}:11311'.format(ROS_MASTER_IP)
print("ROS_MASTER_IP is {}".format(ROS_MASTER_IP))

### Check environment  

It should outputs like this:

```
192.168.10.12
http://192.168.10.12:11311
```

If the result is `http://localhost:11311`, re-run the above procedure.

In [ ]:
# Check Env
!echo $ROS_IP
!echo $ROS_MASTER_URI

## Import libraries

In [ ]:
from sidecar import Sidecar
# Using jupyros(jupyter-ros) 0.5.0 from conda for now
try:
    from jupyros import ros3d
except ImportError:
    from jupyros.ros1 import ros3d

# Check jupyros version
import jupyros
print(jupyros.__version__)

In [ ]:
from tflite_runtime.interpreter import Interpreter
import numpy as np
import cv2
from matplotlib import pyplot as plt

import tflite_runtime
print(tflite_runtime.__version__)

## Load Tensor Flow Lite

In [ ]:
# Load model
interpreter = Interpreter(model_path="coco_ssd_mobilenet_v1_1.0_quant_2018_06_29/detect.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
height = input_details[0]['shape'][1]
width = input_details[0]['shape'][2]
floating_model = (input_details[0]['dtype'] == np.float32)

input_mean = 127.5
input_std = 127.5
# Check output layer name to determine if this model was created with TF2 or TF1,
# because outputs are ordered differently for TF2 and TF1 models
outname = output_details[0]['name']

if ('StatefulPartitionedCall' in outname): # This is a TF2 model
    boxes_idx, classes_idx, scores_idx = 1, 3, 0
else: # This is a TF1 model
    boxes_idx, classes_idx, scores_idx = 0, 1, 2

# un-comment to see the details
# print(input_details, output_details, height, width)

In [ ]:
# Load the label map
with open("coco_ssd_mobilenet_v1_1.0_quant_2018_06_29/labelmap.txt", "r") as f:
    labels = [line.strip() for line in f.readlines()]

# Have to do a weird fix for label map if using the COCO "starter model" from
# https://www.tensorflow.org/lite/models/object_detection/overview
# First label is '???', which has to be removed.
if labels[0] == '???':
    del(labels[0])

In [ ]:
# Check labels
print(len(labels))
print(labels)

## Subscribe camera image

### Start camera image publish node

Before starting subscriber, we need to start the ROS node to publish camera image.  
Open terminal and run `cd ~/dev/mini-pupper-jupyter-notebooks && roslaunch ROS/camera.launch`

<img src="https://i.gyazo.com/4c1d067e131340e4a15792a08204ef22.png" width="50%">

### Check topic

If it shows `ERROR: Unable to communicate with master!` or `ERROR: Unknown topic /image_raw`, re-run the network configuration step.

In [ ]:
!rostopic info /image_raw

### Create Jupyter widgets

First, create an empty widget.

<img src="https://i.gyazo.com/d219da88864a18ecdbc79b8ed939ab4c.png" width="200px">

In [ ]:
from IPython.display import display
import ipywidgets.widgets as widgets


image_widget = widgets.Image(format='jpeg', width=600, height=600)
try:
    sc = Sidecar(title='Object Detection Output')
    with sc:
        display(widgets.VBox([image_widget]))
except:
    display(widgets.VBox([image_widget]))

In [ ]:
import cv2
from cv_bridge import CvBridge, CvBridgeError
from io import BytesIO

import rospy
from sensor_msgs.msg import Image

ros_msg = Image()
bridge = CvBridge()

min_conf_threshold = 0.5

rospy.init_node("image_subscriber")
rate = rospy.Rate(5)

def callback(msg):
    if rate.remaining() > rospy.Duration(0):
        return
    try:
        image = bridge.imgmsg_to_cv2(msg, "bgr8")
    except CvBridgeError as e:
        rospy.logerr("CvBridge Error: {0}".format(e))

    # Load image and resize to expected shape [1xHxWx3]
    imH, imW, _ = image.shape 
    image_resized = cv2.resize(image, (width, height))
    input_data = np.expand_dims(image_resized, axis=0)

    # Normalize pixel values if using a floating model (i.e. if model is non-quantized)
    if floating_model:
        input_data = (np.float32(input_data) - input_mean) / input_std

    # Perform the actual detection by running the model with the image as input
    interpreter.set_tensor(input_details[0]['index'],input_data)
    interpreter.invoke()

    # Retrieve detection results
    boxes = interpreter.get_tensor(output_details[boxes_idx]['index'])[0] # Bounding box coordinates of detected objects
    classes = interpreter.get_tensor(output_details[classes_idx]['index'])[0] # Class index of detected objects
    scores = interpreter.get_tensor(output_details[scores_idx]['index'])[0] # Confidence of detected objects

    # Loop over all detections and draw detection box if confidence is above minimum threshold
    for i in range(len(scores)):
        if ((scores[i] > min_conf_threshold) and (scores[i] <= 1.0)):

            # Get bounding box coordinates and draw box
            # Interpreter can return coordinates that are outside of image dimensions, need to force them to be within image using max() and min()
            ymin = int(max(1,(boxes[i][0] * imH)))
            xmin = int(max(1,(boxes[i][1] * imW)))
            ymax = int(min(imH,(boxes[i][2] * imH)))
            xmax = int(min(imW,(boxes[i][3] * imW)))

            cv2.rectangle(image, (xmin,ymin), (xmax,ymax), (10, 255, 0), 2)

            # Draw label
            object_name = labels[int(classes[i])] # Look up object name from "labels" array using class index
            label = '%s: %d%%' % (object_name, int(scores[i]*100)) # Example: 'person: 72%'
            labelSize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 2.5, 4) # Get font size
            label_ymin = max(ymin, labelSize[1] + 10) # Make sure not to draw label too close to top of window
            cv2.rectangle(image, (xmin, label_ymin-labelSize[1]-10), (xmin+labelSize[0], label_ymin+baseLine-10), (255, 255, 255), cv2.FILLED) # Draw white box to put label text in
            cv2.putText(image, label, (xmin, label_ymin-7), cv2.FONT_HERSHEY_SIMPLEX, 2.5, (0, 0, 0), 4) # Draw label text

    _, frame = cv2.imencode('.jpg', cv2.resize(image, (int(width), int(height))), [int(cv2.IMWRITE_JPEG_QUALITY), 100])
    frame_buffer = BytesIO(frame)
    image_widget.value = frame_buffer.getvalue()

    rate.last_time = rospy.rostime.get_rostime()


Run the below cell to start subscribing image and updating widget.

In [ ]:
import jupyros

jupyros.subscribe("/image_raw", Image, callback)